# **Rule-Based Algorithm**
### Group 1 - Detection of Negation and Uncertainty

- Marino Oliveros Blanco NIU:1668563
- 
- 
- 

**Data loading**

In [12]:
# Libraries
import json
import re
from langdetect import detect # Library for language detection
from spellchecker import SpellChecker
import spacy
import string

In [13]:
# Loading the json file
loading = open("negacio_train_v2024.json",)
training_data = json.load(loading)

**Pre-processing**
- Removing redacted entries
- Removing language mixes
- Solving misspelled words
- Removing patient information
- Tokenizing 

In [14]:
# 1 Remove pacient information and redacted entries
def remove_pacient_info(text):
    # Remove lines starting with "nº historia clinica:" and ending with "motiu d'ingres"
    text = re.sub(r'nº historia clinica:.*?motiu d\'ingres', '', text, flags=re.DOTALL)
    # Remove all asterisks '*'
    text = text.replace('*', '')
    return text


# 2 Remove Punctuation (Able to be turned ON/OFF)
def remove_punctuation(text):
    # Define a translation table to map punctuation to None
    translator = str.maketrans('', '', string.punctuation)
    # Remove punctuation using the translation table
    text = text.translate(translator)
    return text


# 3 Spell cheking with language detection (Able to be turned ON/OFF)
# Load language models for Spanish and Catalan
nlp_es = spacy.load("es_core_news_sm")
nlp_ca = spacy.load("ca_core_news_sm")

def spell_check_and_lemmatize(text):
    # Detect the language of the text
    language = detect(text)
    # Initialize spell checker
    spell = SpellChecker(language='es')  # As most of the text is in Spanish

    # Tokenize the text using the appropriate language model
    if language == 'ca':
        doc = nlp_ca(text)
    else:
        doc = nlp_es(text)

    # Correct misspelled words and lemmatize tokens
    corrected_tokens = []
    for token in doc:
        # Check if the token is a punctuation or whitespace
        if not token.is_punct and not token.is_space:
            # Get the corrected version of the token
            corrected_token = spell.correction(token.text)
            # Lemmatize the corrected token
            corrected_token_lemma = token.lemma_ if token.lemma_ != '-PRON-' else corrected_token
            corrected_tokens.append(corrected_token_lemma)
        else:
            corrected_tokens.append(token.text)

    # Join the tokens back into text
    corrected_text = ' '.join(corrected_tokens)
    return corrected_text


# 4 Tokenization with coordinates of the original text for (evaluation)
def tokenize_with_coordinates(text):
    # Tokenize the text while preserving the coordinates
    tokens_with_coordinates = []
    token_start = 0
    for token in re.finditer(r'\S+', text):
        token_text = token.group(0)
        token_end = token_start + len(token_text)
        tokens_with_coordinates.append((token_text, token_start, token_end))
        # Update token start position for the next token
        token_start = token_end
    return tokens_with_coordinates


# Main function to process the text
def pre_process_text(text, remove_punctuation_call=True, spell_check_call=True):
    
    # 1 Remove pacient information and redacted entries
    preprocessed_text = remove_pacient_info(text)

    # 2 Remove punctuation if specified
    if remove_punctuation_call:
        preprocessed_text = remove_punctuation(preprocessed_text)

    # 3 Spell check and lemmatize if specified
    if spell_check_call:
        preprocessed_text = spell_check_and_lemmatize(preprocessed_text)

    # 4 Tokenize the text with coordinates
    tokens_with_coordinates = tokenize_with_coordinates(preprocessed_text)

    return tokens_with_coordinates

Sample Usage

In [15]:
# Example usage with string:
# Let's assume the text we want to process is stored in the variable `text`
text = "nº historia clinica: ** *** *** nºepisodi: ******** sexo: home data de naixement: 16.05.1936 edad: 82 anys procedencia cex mateix hosp servei urologia data d'ingres 24.07.2018 data d'alta 25.07.2018 08:54:04 ates per ***************, *****; ****************, ****** informe d'alta d'hospitalitzacio motiu d'ingres paciente que ingresa de forma programada para realizacion de uretrotomia interna . antecedents alergia a penicilina y cloramfenicol . no habitos toxicos. antecedentes medicos: bloqueo auriculoventricular de primer grado hipertension arterial. diverticulosis extensa insuficiencia renal cronica colelitiasis antecedentes quirurgicos: exeresis de lesiones cutaneas con anestesia local protesis total de cadera cordectomia herniorrafia inguinal proces actual varon de 81a que a raiz de episodio de hematuria macroscopica se realiza cistoscopia que es negativa para lesiones malignas pero se objetiva estenosis de uretra . se intentan dilataciones progresivas en el gabinete de urologia sin exito. se solicita estudio de imagen que confirma la existencia de estenosis a nivel d uretra bulbar por lo que se indica uretrtomia interna. exploracio complementaria uretrocistografia retrograda + cums (11/2017): la uretrografia retrograda muestra una uretra anterior con dos estenosis focales a nivel de uretra peneana y bulbar, aunque se observa paso de contraste retrogrado a vejiga. vejiga de correcta capacidad (250 cc de contraste), de paredes trabeculadas y con diverticulos, el mayor de ellos en cara posterolateral izquierda, sin observarse defectos de replecion. la uretrografia miccional muestra una uretra prostatica dilatada, sin claras estenosis focales confirmandose la existencia de las dos estenosis de uretra anterior descritas previamente. moderado residuo postmiccional en vejiga asi como en el interior del diverticulo posterolateral izquierdo descrito. uretroscopia (10/2017) falsa via a nivel de uretra peneana, siguiendo la uretra se detecta gran estenosis que no permite el paso de una guia. nhc ** *** *** (********) age-v-uro 1/2 lopd evolucio clinica el 24 de julio de 2018 con el consentimiento informado del paciente y sin contraindicacion preoperatoria se realiza uretrotomia interna sin incidencias. tras el procedimiento el paciente es trasladado a la planta de hospitalizacion siendo portador de lavado vesical continuo. posteriormente se mantiene en buen estado general, afebril, hemodinamicamente estable y con buen control del dolor. aclarado progresivo de la orina con los lavados vesicales continuos, que permiten su retirada, conserva correcta diuresis. tolerancia correcta a dieta oral. dada la buena evolucion se decide alta domiciliaria siendo portador de sonda vesical. orientacio diagnostica n40.0 hiperplasia prostatica benigna sense simptomes en les vies urinaries inferiors procediments 04.81 injeccio en el nervi periferic d'anestesic per a analgesia 58.0 uretrotomia. excisio de septe uretral, uretrostomia perineal, extraccio de calcul uretral per incisio sonda vesical profilaxis antibiotica, antilucerosa y antitrombotica tractament i recomanacions a l'alta -abundante ingesta de liquidos entorno a dos litros y medio de agua al dia. -puede orinar con restos de sangre durante las proximas semanas. -es normal que sienta escozor al orinar y que tenga algun escape de orina y urgencia miccional al retirar la sonda vesical. mantener sonda vesical durante 14 dias (dos semanas). ciprofloxacino 500mg cada 12h durante dos semanas. -paracetamol 1 g cada 8 horas si molestias. -si fiebre mayor de 38ºc, empeoramiento claro del estado general o imposibilidad miccional por obstruccion de sonda vesical o despues de su retirada, consultar con el servicio de urgencias. -control en consultas externas de urologia segun cita en hoja adjunta. destinacio a l'alta: a domicili nhc ** *** *** (********) age-v-uro 2/2 lopd"
processed_text = pre_process_text(text, remove_punctuation_call=True, spell_check_call=True)
for token, start, end in processed_text:
    print(f"Token: {token}, Start: {start}, End: {end}")

Token: paciente, Start: 0, End: 8
Token: que, Start: 8, End: 11
Token: ingresar, Start: 11, End: 19
Token: de, Start: 19, End: 21
Token: forma, Start: 21, End: 26
Token: programado, Start: 26, End: 36
Token: para, Start: 36, End: 40
Token: realizacion, Start: 40, End: 51
Token: de, Start: 51, End: 53
Token: uretrotomia, Start: 53, End: 64
Token: interno, Start: 64, End: 71
Token: antecedents, Start: 71, End: 82
Token: alergiar, Start: 82, End: 90
Token: a, Start: 90, End: 91
Token: penicilina, Start: 91, End: 101
Token: y, Start: 101, End: 102
Token: cloramfenicol, Start: 102, End: 115
Token: no, Start: 115, End: 117
Token: habito, Start: 117, End: 123
Token: toxico, Start: 123, End: 129
Token: antecedente, Start: 129, End: 140
Token: medico, Start: 140, End: 146
Token: bloqueo, Start: 146, End: 153
Token: auriculoventricular, Start: 153, End: 172
Token: de, Start: 172, End: 174
Token: primero, Start: 174, End: 181
Token: grado, Start: 181, End: 186
Token: hipertension, Start: 186, End

Real Usage

In [16]:
for entry in training_data:
    text = entry["data"]["text"]  # Extract the text from the JSON object
    processed_text = pre_process_text(text, remove_punctuation_call=True, spell_check_call=True)
    for token, start, end in processed_text:
        print(f"Token: {token}, Start: {start}, End: {end}")

Token: paciente, Start: 0, End: 8
Token: que, Start: 8, End: 11
Token: ingresar, Start: 11, End: 19
Token: de, Start: 19, End: 21
Token: forma, Start: 21, End: 26
Token: programado, Start: 26, End: 36
Token: para, Start: 36, End: 40
Token: realizacion, Start: 40, End: 51
Token: de, Start: 51, End: 53
Token: uretrotomia, Start: 53, End: 64
Token: interno, Start: 64, End: 71
Token: antecedents, Start: 71, End: 82
Token: alergiar, Start: 82, End: 90
Token: a, Start: 90, End: 91
Token: penicilina, Start: 91, End: 101
Token: y, Start: 101, End: 102
Token: cloramfenicol, Start: 102, End: 115
Token: no, Start: 115, End: 117
Token: habito, Start: 117, End: 123
Token: toxico, Start: 123, End: 129
Token: antecedente, Start: 129, End: 140
Token: medico, Start: 140, End: 146
Token: bloqueo, Start: 146, End: 153
Token: auriculoventricular, Start: 153, End: 172
Token: de, Start: 172, End: 174
Token: primero, Start: 174, End: 181
Token: grado, Start: 181, End: 186
Token: hipertension, Start: 186, End

Cosas a notar: hace de texto en texto, por todo el json

**Implementation**

In [ ]:

# We can use any of the following methods to detect negation or uncertainty in a sentence

#Pre-defined list of negation trigger words
#Regular expressions
#Part of Speech tagging
#Syntactic parsing

**Results**

In [ ]:
# Explore the results